In [ ]:
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install accelerate>=0.21.0


In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments,Trainer
import evaluate

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv('/content/drive/MyDrive/Data Science/EMAIL SPAM DETECTION/DATA/cleaned_email_message_based.csv')
data

Mounted at /content/drive


In [6]:
data=pd.read_csv('/content/drive/MyDrive/Data Science/EMAIL SPAM DETECTION/DATA/cleaned_email_message_based.csv')
data

,spam,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5566,1,This is the 2nd time we have tried 2 contact u...
5567,0,Will ü b going to esplanade fr home?
5568,0,"Pity, * was in mood for that. So...any other s..."
5569,0,The guy did some bitching but I acted like i'd...


In [34]:
x_train,x_test,y_train,y_test=train_test_split(data['Message'],data['spam'],test_size=0.2)

In [7]:
train_dataset=Dataset.from_pandas(data.sample(4000,random_state=42))
eval_dataset=Dataset.from_pandas(data.sample(200,random_state=30))
test_dataset=Dataset.from_pandas(data.sample(1000,random_state=20))

In [8]:
train_dataset

Dataset({
    features: ['spam', 'Message', '__index_level_0__'],
    num_rows: 4000
})

In [9]:
def tokenizer_function(x):
  return tokenizer(x['Message'],padding="max_length",truncation=True)
tokenized_train=train_dataset.map(tokenizer_function)
tokenized_eval=eval_dataset.map(tokenizer_function)
tokenized_test=test_dataset.map(tokenizer_function)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
metric = evaluate.load("accuracy")

In [18]:
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
)

In [16]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)
def custom_loss_fn(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
predictions_output = trainer.predict(tokenizer_datasets_test)
accuracy_score = compute_metrics((predictions_output.predictions,tokenizer_datasets_test['label']))
print(accuracy_score)

In [9]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv('/content/drive/MyDrive/Data Science/EMAIL SPAM DETECTION/DATA/cleaned_email_message_based.csv')
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
data

In [21]:
x_train,x_test,y_train,y_test=train_test_split(data['Message'],data['spam'],test_size=0.2)
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [22]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)
history=model.fit(x_train, y_train, epochs=1,batch_size=32,validation_data=(x_test,y_test),verbose=1)

In [27]:
print(history.history)

{'loss': [0.3340279459953308], 'accuracy': [0.8610861897468567], 'precision': [0.36486485600471497], 'recall': [0.045075125992298126], 'val_loss': [0.2753480076789856], 'val_accuracy': [0.8681614398956299], 'val_precision': [0.5], 'val_recall': [0.013605441898107529]}


In [28]:
loss=history.history['loss']
loss

[0.3340279459953308]